In [228]:
import pandas as pd
import pymongo
import numpy as np


pd.set_option("display.max_rows", 300)

### Extract and Clean Data

#### Population of Cities

In [229]:
# url to scrape for the city population
cities_url ="https://worldpopulationreview.com/world-cities"
# Use pandas `read_html` to parse the url
df_cityPop = pd.read_html(cities_url, header=0)[0]
# rename the columns
df_cityPop.rename(columns={'Name':'City', 
                           '2020 Population':'2020',
                           '2019 Population':'2019'
                          },inplace=True)
# Replace null values with 0
df_cityPop.fillna(0,inplace = True)
#  converting column 2019 from float to int
df_cityPop['2019'] = df_cityPop['2019'].apply(np.int32)

df_cityPop.head()

,Rank,City,Country,2020,2019,Change
0,1,Tokyo,Japan,37393128,37435192,-0.11%
1,2,Delhi,India,30290936,29399140,3.03%
2,3,Shanghai,China,27058480,26317104,2.82%
3,4,Sao Paulo,Brazil,22043028,21846508,0.90%
4,5,Mexico City,Mexico,21782378,21671908,0.51%


#### Live Population Data Scraped

In [230]:
# url to scrape for the Live population data
countries_url ="https://worldpopulationreview.com"
# Use pandas `read_html` to parse the url
df_LatestPop = pd.read_html(countries_url, header=0)[0]
# eliminating unnessasary data
df_LatestPop = df_LatestPop.iloc[:,[1,2,4,5,6,7,8]]
# rename the columns
df_LatestPop.rename(columns={'Area':'Area_SqKm',
                             '2019 Density':'Density_PerSqKm',
                             'Growth Rate':'Growth_Percentage', 
                             'World %':'World_Percentage'
                            },inplace=True)
                            
# Converting string values to numbers
df_LatestPop['Area_SqKm'] = pd.to_numeric(df_LatestPop['Area_SqKm'].str.rsplit(' ', 0).str.get(0).str.replace(r',', ''))
df_LatestPop['Density_PerSqKm'] = pd.to_numeric(df_LatestPop['Density_PerSqKm'].str.rsplit('/', 0).str.get(0).str.replace(r',', ''))
df_LatestPop['Growth_Percentage'] = pd.to_numeric(df_LatestPop['Growth_Percentage'].str.rsplit('%', 0).str.get(0))
df_LatestPop['World_Percentage'] = pd.to_numeric(df_LatestPop['World_Percentage'].str.rsplit('%', 0).str.get(0))
df_LatestPop.head()

,Country,2021 (Live),Area_SqKm,Density_PerSqKm,Growth_Percentage,World_Percentage,Rank
0,China,1442166775,9706961,148,0.39,18.47,1
1,India,1387177258,3287590,420,0.99,17.70,2
2,United States,332041150,9372610,35,0.59,4.25,3
3,Indonesia,275056196,1904569,144,1.07,3.51,4
4,Pakistan,223154088,881912,250,2.00,2.83,5


#### Scraping Country codes to merge datasets with

In [231]:
# url to scrape for ISO 3166 country codes Alpha-2 and Alpha-3 from www.iban.com
country_code_url ="https://www.iban.com/country-codes"
# Use panda's `read_html` to parse the url
df_countryCode = pd.read_html(country_code_url, header=0)[0]
# eliminating unnessasary data
df_countryCode = df_countryCode.iloc[:,[1,2]]
# rename the columns
df_countryCode.rename(columns={'Alpha-2 code':'Country_Code_2',
                               'Alpha-3 code':'Country_Code'
                              },inplace=True)
df_countryCode.head()

,Country_Code_2,Country_Code
0,AF,AFG
1,AX,ALA
2,AL,ALB
3,DZ,DZA
4,AS,ASM


#### Population of Countries

In [232]:
# read Countries population data from csv(source:https://worldpopulationreview.com) into dataframe
df_countries = pd.read_csv('static/data/csvData.csv')
# rename the columns
df_countries.rename(columns={'cca2':'Country_Code_2',
                             'name':'Country',
                             'pop2050':'2050',
                             'pop2030':'2030',
                             'pop2020':'2020',
                             'pop2019':'2019',
                             'pop2015':'2015',
                             'pop2010':'2010',
                             'pop2000':'2000',
                             'pop1990':'1990',
                             'pop1980':'1980',
                             'pop1970':'1970' 
                            },inplace=True)

# # eliminating unnessasary data
df_countries = df_countries.iloc[:,[0,1,4,5,2,3,6,7,8,9,10,11]]

# Removing decimal point from data
# Loop through the columns
for col in df_countries:
    # performing operations on columns other than Country column
    if col not in ["Country_Code_2", "Country"]:
        # correcting the decimal positions
        df_countries[col] = (df_countries[col] * 1000).astype(int)

df_countries.head()

,Country_Code_2,Country,2050,2030,2020,2019,2015,2010,2000,1990,1980,1970
0,CN,China,1402405170,1464340159,1439323776,1433783686,1406847870,1368810615,1290550765,1176883674,1000089235,827601394
1,IN,India,1639176033,1503642322,1380004385,1366417754,1310152403,1234281170,1056575549,873277798,698952844,555189792
2,US,United States,379419102,349641876,331002651,329064917,320878310,309011475,281710909,252120308,229476354,209513341
3,ID,Indonesia,330904664,299198430,273523615,270625568,258383256,241834215,211513823,181413402,147447836,114793178
4,PK,Pakistan,338013196,262958794,220892340,216565318,199426963,179424641,142343578,107647921,78054343,58142060


In [233]:
# Another Dataset
# Cleaning csv Population data from https://datacatalog.worldbank.org
# reading csv's into dataframes
df_population = pd.read_csv('static/data/population.csv')

# Function to Clean each dataframes
def clean_dataFrames(df, col_list):
    # eliminating unnecessary data
    df = df.iloc[0:217, col_list]
    # renaming columns
    df.rename(columns= {df.columns[0]: "Name"}, inplace = True)
    df = df.rename(columns = lambda x : (str(x)[:-9]))
    df.rename(columns= {df.columns[0]: "Country", df.columns[1]: "Country_Code"}, inplace = True)
    return df

# list of required column indexes
col_list = [2,3,11,12,13]
# Calling clean_dataFrames function passing the dataframe as parameter
df_population = clean_dataFrames(df_population, col_list)

# Removing row with no values for the required years(Country Eritrea)
df_population.drop(df_population.index[df_population['Country'] == 'Eritrea'], inplace = True)

# Loop through the columns to covert values from string to 
for col in df_population:
    # on columns other than Country and Country_Code_2
    if col not in ["Country_Code", "Country"]:
        # Converting string to number
        df_population[col] = df_population[col].astype(float).apply(np.int32)

df_population.head()

,Country,Country_Code,2016,2017,2018
0,Afghanistan,AFG,35383128,36296400,37172386
1,Albania,ALB,2876101,2873457,2866376
2,Algeria,DZA,40551404,41389198,42228429
3,American Samoa,ASM,55741,55620,55465
4,Andorra,AND,77297,77001,77006


In [234]:
# merging two dataframes for additional years data

# merging df_population with df_countryCode
df_population = df_countryCode.merge(df_population, on="Country_Code", how="right")



# merging df_population with df_countries
df_countries = df_countries.merge(df_population, on="Country_Code_2", how="left")
# removing duplicated Country column and Country_Code_2
df_countries = df_countries.drop(['Country_y', 'Country_Code_2'], axis=1)
# renaming columns
df_countries.rename(columns= {"Country_x": "Country"}, inplace = True)
# reordering the columns
df_countries = df_countries.iloc[:,[0,11,1,2,3,4,5,14,13,12,6,7,8,9,10]]
# Replace null values with 0
df_countries.fillna(0,inplace = True)
# converting float values to int
df_countries[['2016','2017','2018']] = df_countries[['2016','2017','2018']].apply(np.int32)

df_countries.head()

,Country,Country_Code,2050,2030,2020,2019,2015,2018,2017,2016,2010,2000,1990,1980,1970
0,China,CHN,1402405170,1464340159,1439323776,1433783686,1406847870,1392730000,1386395000,1378665000,1368810615,1290550765,1176883674,1000089235,827601394
1,India,IND,1639176033,1503642322,1380004385,1366417754,1310152403,1352617328,1338658835,1324509589,1234281170,1056575549,873277798,698952844,555189792
2,United States,USA,379419102,349641876,331002651,329064917,320878310,326687501,324985539,322941311,309011475,281710909,252120308,229476354,209513341
3,Indonesia,IDN,330904664,299198430,273523615,270625568,258383256,267663435,264645886,261554226,241834215,211513823,181413402,147447836,114793178
4,Pakistan,PAK,338013196,262958794,220892340,216565318,199426963,212215030,207896686,203627284,179424641,142343578,107647921,78054343,58142060


### Load Data into MongoDB

In [235]:
# Loading Data into MongoDB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db_name = "populationDB"
# # Drop database if exists
if bool(db_name in client.list_database_names()):
    client.drop_database(db_name)

# Creating Database and collection in mongodb
db = client[db_name]
countriesPop = db["countriesPopulation"]
citiesPop = db["citiesPopulation"]
latestPop = db["latestPopulation"]


# Function to insert Dataframes into mongodb collections
def insertToDB(df, collection):
    data_dict = df.to_dict("records") # Convert to dictionary
    # removing index from data
    data_dict = [{k: v for k, v in d.items() if k != 'index'} for d in data_dict]
    collection.insert_one({"data":data_dict}) # Insert dict to collection


# Calling function to insert each dataframes into mongoDB collections
insertToDB(df_countries, countriesPop)
insertToDB(df_cityPop, citiesPop)
insertToDB(df_LatestPop, latestPop)


print(db.list_collection_names())

['latestPopulation', 'countriesPopulation', 'citiesPopulation']
